In [3]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

from pandas import read_csv
import pandas as pd
import datetime as dt
import matplotlib
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import csv
import tensorflow as tf
import keras
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from keras.regularizers import l2
from tensorflow.keras import optimizers
from keras.callbacks import TerminateOnNaN
from tensorflow import keras
from tensorflow.keras import layers


path1 = '/content/drive/MyDrive/Master_thesis/Datasets/Australia_numpy_removed_night_hours.npy'
path2 = '/content/drive/MyDrive/Master_thesis/Datasets/irradiance_AUS(2012-2013).csv'

Mounted at /content/drive


In [4]:
n_house = 100
valid_house = 20 
test_house = 20  #for final predictions 

In [5]:
import random
np.random.seed(42)

houses = np.loadtxt(path1)
shuffle = np.arange(houses.shape[1])
np.random.shuffle(shuffle)
houses = houses[:,shuffle]
irraidance=read_csv(path2, header =0, index_col=0) 
irraidance=irraidance.drop("time", axis=1)
irradiance_array= irraidance.values
weath_data = irradiance_array.reshape(irradiance_array.shape[0]*irradiance_array.shape[1])
train_val_set = houses[:,:n_house+valid_house]
train_val_set = train_val_set.reshape(train_val_set.shape[0]*train_val_set.shape[1], order="F")
weath_data_stacked = np.tile(weath_data,n_house+valid_house)
data2 = pd.DataFrame(np.column_stack((train_val_set,weath_data_stacked)))
test_set = houses[:,-20:] #don't stack. Keep aside for final predictions and loop house by house (add weath_data in each loop)
print(shuffle)

[281 265 164   9  77 278  93 109   5 173  97 195 184 154  57  60 147 108
  63 140 155 104 247  46  42 275 280 116 213 236  17 239  33  24  45   7
 113 194 111  92  75  82 118  76 129 197 210 288 219 178 144 186  84 248
 277  73 244  25 209  59   6 183 185 146  30  22 254  56 237 285 126 228
 283 255 158 225  78  66 296 192 181  19 170  79  90 132  72  15  10 157
 224  68 222  37  16 119 268 269  67 101  69  31 172 148 223 114  18 179
  96 165 291  86 246 143 231 124 139 152 204  55 232 182 180 137 251  38
 125 193 167 203 112 264 117 282 273 221 176   2 115 177 175 120 208 259
 127  74  29  83 261 107 287 245 250 294 230  65 196  85 211 159  12  35
  28 142 229 279 168  51  95 206 218  41  89 215 136  26 292 141 198   0
 267 271 100 258 253 171  98  36  61 150 234 200 240  11 295 266  27 242
   4 122  32 202 162 226 256 138  62 135 128 289   8  70 263  64  44 233
 156  40 123 274 216 153  23 260 110  81 207 212  39 238 290 284 199  14
  47  94 262 227 272 201 161  43 217 145 190 220 24

In [6]:
# # split a multivariate dataset into train/test sets
def split_dataset(data,test_houses):
	# split into standard weeks
 train, test = data[0:-365*18*test_houses], data[-365*18*test_houses:]
	# restructure into windows of weekly data
 train = array(split(train, len(train)/18))
 test = array(split(test, len(test)/18))
 print(train.shape,test.shape)
 return train, test
 
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores
 
# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))
 
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=18):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)


def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res



In [20]:
def build_model(train,n_input,batch_size,epochs,head_size=128,num_heads=4,ff_dim=4,num_transformer_blocks=4,mlp_units=[128],dropout=0.1,mlp_dropout=0):
  train_x, train_y = to_supervised(train, n_input=n_input)
  input_shape = train_x.shape[1:]
  print("Train shape =", train_x.shape)
  print("Input shape =", input_shape)
  inputs = keras.Input(shape=input_shape)
  x = inputs
  for _ in range(num_transformer_blocks):
     x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
  x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
  for dim in mlp_units:
     x = layers.Dense(dim, activation="relu")(x)
     x = layers.Dropout(mlp_dropout)(x)
  outputs = layers.Dense(18, activation="relu")(x)
  model =  keras.Model(inputs, outputs)
  model.compile(loss="mse",optimizer=keras.optimizers.Adam(learning_rate=0.0001),metrics='MeanSquaredError')
  #  # stop when loss remains the same for 3 epochs --> passed to the model.fit
  early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, mode='min')
  #saves the model on drive.
  path = '/content/drive/MyDrive/Master_thesis/Models/Transformer/LR_0.0001_MLP_128_Head_size_128_E'+str(epochs)+'_B'+str(batch_size)+'_house_'+str(n_house)+'_input_'+str(n_input)+'.h5'
  model_checkpoint = ModelCheckpoint(path, monitor='loss', mode='min', verbose=1, save_best_only=True)
  #stop training when loss gets to Nan
  call = keras.callbacks.TerminateOnNaN()
	#model.fit(x_train,y_train,validation_split=0.2,epochs=0,batch_size=64,callbacks=callbacks)
  model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,callbacks=[early_stopping,model_checkpoint,call])
  return model.summary(),model


In [ ]:
# make a forecast
def forecast(model, history,  n_input):
  # flatten data
  data = array(history)
  data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
  # retrieve last observations for input data
  input_x = data[-n_input:, :]
  # reshape into [samples, timesteps, rows, cols, channels]
  input_x = input_x.reshape((1, n_input, 2))
  # forecast the next week
  yhat = model.predict(input_x, verbose=0)
  # we only want the vector forecast
  yhat = yhat[0]
  return yhat


# define the total days to use as input
n_input = 18
#define parameters for the model
epochs, batch_size = 30,32
#data preparation
data= data2.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(data) 
train,test = split_dataset(data,valid_house)
train_scaled, test_scaled = split_dataset(scaled,valid_house)
test_X,test_y = to_supervised(test_scaled,n_input)


# evaluate model and get scores
#model,history_model = build_model(train_scaled, test_X, test_y, epochs, batch_size, n_steps, n_length, n_input)

model_summary, model = build_model(train_scaled,n_input,batch_size,epochs)

history = [x for x in train_scaled]
predictions = list()

for i in range(len(test)):
  yhat_sequence = forecast(model, history,  n_input)
  #inverse scaling manually 
  test_min = np.min(data[-365*18*valid_house:,0])
  test_max = np.max(data[-365*18*valid_house:,0])
  yhat_sequence = yhat_sequence * (test_max-test_min)
  predictions.append(yhat_sequence)
  #add actual observations for next forecasting
  history.append(test_scaled[i, :])
predictions = array(predictions)
score, scores = evaluate_forecasts(test[:, :, 0], predictions)

#summarize scores
summarize_scores('ConvLSTM', score, scores)
# plot scores
hours = ['1', '2', '3', '4', '5', '6', '7','8','9','10','11','12','13','14','15','16','17','18']
pyplot.plot(hours, scores, marker='o', label='ConvLSTM')
pyplot.show()

# plot history
# pyplot.plot(history_model['loss'], label='train')
# pyplot.plot(history_model['val_loss'], label='test')
# pyplot.legend()
# pyplot.show()

(36500, 18, 2) (7300, 18, 2)
(36500, 18, 2) (7300, 18, 2)
Train shape = (656965, 18, 2)
Input shape = (18, 2)
Epoch 1/30
20530/20531 [============================>.] - ETA: 0s - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 00001: loss improved from inf to 0.00494, saving model to /content/drive/MyDrive/Master_thesis/Models/Transformer/LR_0.0001_MLP_128_Head_size_128_E30_B32_house_100_input_18.h5
20531/20531 [==============================] - 1852s 90ms/step - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 2/30
20530/20531 [============================>.] - ETA: 0s - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 00002: loss improved from 0.00494 to 0.00480, saving model to /content/drive/MyDrive/Master_thesis/Models/Transformer/LR_0.0001_MLP_128_Head_size_128_E30_B32_house_100_input_18.h5
20531/20531 [==============================] - 1844s 90ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 3/30
20530/20531 [============================>.] - ETA: 0s - loss: 0.0047 - mean_

#Predicting from the saved model

In [10]:

def forecast_daily(model, test,weather,input_length):
  all = list()
  for house in range(test.shape[1]):
    predictions = list()
    #iterating over houses and taking each house seperately
    data = test[:,house]
    #finding min and max values each house seperately
    house_max = np.max(data)
    house_min = np.min(data)
    #combining house with weather data
    data = np.column_stack((data,weather))
    #scale
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)
    for hour in range(0,data.shape[0],18):
      #looping thourough each house - moving window
      input = data[hour:hour+input_length,:]
      if input.shape[0]*input.shape[1] < 2* input_length:
        break
      input = input.reshape((1, n_input, 2))
      y_hat = model.predict(input)
      y_hat = np.array(y_hat).flatten()
      #inverse scaling
      y_hat = y_hat * (house_max-house_min)
      predictions.append(y_hat)
      #save predictions
    with open('/content/drive/MyDrive/Master_thesis/Results/Transformer/E20_B64_'+str(house)+'_input_'+str(n_input)+'.csv','a') as nd:
      write_n = csv.writer(nd)
      write_n.writerows(predictions)
    all.append(predictions)
  return np.array(all).shape

# define the total days to use as input
#n_input = 18

model = keras.models.load_model('/content/drive/MyDrive/Master_thesis/Models/Transformer/E20_B64_'+str(n_house)+'_input_'+str(90)+'.h5')
forecast_daily(model,test_set,weath_data,n_input)

(20, 365, 18)

#Evaluation

In [15]:
### EVALUATION ### 

print("Shape of the test set = ", test_set.shape)
print("Shape of the weather set = ", weath_data.shape)
# define the number of subsequences and the length of subsequences

# define the total days to use as input
#n_input = 18
# substract hours with input length from the beginning of the data
data = test_set[n_input:,:]
from statistics import mean

RMSEs = list()
MAEs=list()


for house in range(data.shape[1]):
  actual_house = data[:,house]
  path ='/content/drive/MyDrive/Master_thesis/Results/Transformer/E20_B64_'+str(house)+'_input_'+str(n_input)+'.csv'
  pred_house = read_csv(path,header=None)[:-1]
  pred_house = np.array(pred_house)
  #print(actual_house.shape,pred_house.shape)
  
  row_pred = 0
  for hours in range(0,data.shape[0]-n_input,18):
  #for hours in range(1):
    actual =  actual_house[hours: 18+hours]
    pred = pred_house[row_pred,:]
 
    rmse = sqrt(mean_squared_error(actual,pred))
    RMSEs.append(rmse)
    mae= mean_absolute_error(actual,pred) 
    MAEs.append(mae)
    row_pred+=1

print("RMSE",mean(RMSEs),"Shape of RMSEs = ", np.array(RMSEs).shape)
print("MAE",mean(MAEs),"Shape of MAEs = ", np.array(MAEs).shape)


Shape of the test set =  (6570, 20)
Shape of the weather set =  (6570,)
RMSE 0.3765129430079039 Shape of RMSEs =  (7260,)
MAE 0.327015023379639 Shape of MAEs =  (7260,)
